In [3]:
def skinDetection(sourceImage):
    # Convert image to YCrCb
    imageYCrCb = cv2.cvtColor(sourceImage,cv2.COLOR_RGB2YCR_CB)

    # Find region with skin tone in YCrCb image
    skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)

    # Do contour detection on skin region
    contours, hierarchy = cv2.findContours(skinRegion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contour on the source image
    max_contour_area = -1
    max_contour_index = -1
    for i, c in enumerate(contours):
        area = cv2.contourArea(c)
        if area > max_contour_area:
            max_contour_index = i
            max_contour_area = area
    sourceImage = numpy.zeros(sourceImage.shape).astype(sourceImage.dtype)
    if max_contour_index > -1: 
        cv2.drawContours(sourceImage, contours, max_contour_index, (255, 255, 255), -1)
    return sourceImage

In [5]:
import cv2
import numpy

haar_file = 'haarcascade_frontalface_default.xml'
 
 
# defining the size of images
(width, height) = (130, 100)   

face_cascade = cv2.CascadeClassifier(haar_file)


# Constants for finding range of skin color in YCrCb
min_YCrCb = numpy.array([0,133,77],numpy.uint8)
max_YCrCb = numpy.array([255,173,127],numpy.uint8)

# Create a window to display the camera feed
cv2.namedWindow('Camera Output')

# Get pointer to video frames from primary device
videoFrame = cv2.VideoCapture(0)

# Process the video frames
keyPressed = -1 # -1 indicates no key pressed

while(keyPressed < 0): # any key pressed has a value >= 0

    # Grab video frame, decode it and return next video frame
    readSucsess, sourceImage = videoFrame.read()
    
    gray = cv2.cvtColor(sourceImage, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 4)
    l = 0
    if len(faces)>0:
        (x, y, w, h) = faces[0]

        sourceImage = sourceImage[y:y + h, x:x + w]

        sourceImage = cv2.cvtColor(sourceImage,cv2.COLOR_BGR2RGB)
        sourceImage = skinDetection(sourceImage)
        
        
        # Display the source image
        cv2.imshow('Camera Output',sourceImage)

    # Check for user input to close program
    keyPressed = cv2.waitKey(1) # wait 1 milisecond in each iteration of while loop

# Close window and camera after exiting the while loop
cv2.destroyWindow('Camera Output')
videoFrame.release()